# Static Analysis of Cantilever Beam

This notebook performs static structural analysis using FEniCS on the imported mesh.

In [41]:
import sys
sys.path.append('scripts')

# Import the static analysis function with material properties
from static_3d import static_analysis_with_materials

In [42]:
import json
import os

# Load analysis configuration from JSON file
config_file = "simulation_config/study_config.json"

with open(config_file, 'r') as f:
    config = json.load(f)

# Get mesh file path and derive other file paths from its location
mesh_file = config["mesh"]["mesh_file"]
mesh_dir = os.path.dirname(mesh_file)

# Parse boundaries - find the fixed boundary (Displacement Constraints)
fixed_boundary = None
for boundary in config["boundaries"]:
    if boundary["definition"]["type"] == "Displacement Constraints":
        fixed_boundary = boundary
        break

if fixed_boundary is None:
    raise ValueError("No fixed boundary (Displacement Constraints) found in configuration")

fixed_marker_name = fixed_boundary["assigned"][0]  # e.g., "Fixed end"

# Parse loads - extract the force load definition
force_load = None
for load in config["loads"]:
    if "FX" in load["definition"]:  # Identify force load
        force_load = load
        break

if force_load is None:
    raise ValueError("No force load found in configuration")

forced_marker_name = force_load["assigned"][0]  # e.g., "Forced end"
f_surface = (
    float(force_load["definition"]["FX"]),
    float(force_load["definition"]["FY"]),
    float(force_load["definition"]["FZ"])
)

# Parse materials - extract material properties
material = None
for mat in config["materials"]:
    if mat["properties"]["type"] == "Linear Elastic":
        material = mat
        break

if material is None:
    raise ValueError("No Linear Elastic material found in configuration")

material_marker_name = material["assigned"][0]  # e.g., "Domain"
E = float(material["properties"]["E"])
nu = float(material["properties"]["nu"])

# Flatten into function parameters
analysis_config = {
    # File paths
    "mesh_file": mesh_file,
    "surface_markers_file": os.path.join(mesh_dir, "surface.xdmf"),
    "solid_markers_file": os.path.join(mesh_dir, "solid.xdmf"),
    "markers_json_file": os.path.join(mesh_dir, "markers.json"),
    "output_file": "post-processing/output/cantilever_results.xdmf",
    
    # Boundary condition marker names
    "fixed_marker_name": fixed_marker_name,
    "forced_marker_name": forced_marker_name,
    "material_marker_name": material_marker_name,
    
    # Material properties
    "E": E,
    "nu": nu,
    
    # Loading conditions
    "f_body": (0.0, 0.0, 0.0),  # No body forces
    "f_surface": f_surface,
    
    # Output control
    "print_results": True
}

print(f"Configuration loaded successfully:")
print(f"  Fixed boundary: '{fixed_marker_name}'")
print(f"  Forced boundary: '{forced_marker_name}'")
print(f"  Material domain: '{material_marker_name}'")
print(f"  Material: E={E}, nu={nu}")
print(f"  Surface force: {f_surface}")


Configuration loaded successfully:
  Fixed boundary: 'fixed_end'
  Forced boundary: 'forced_end'
  Material domain: 'Domain'
  Material: E=2000000.0, nu=0.3
  Surface force: (1.0, 0.0, 0.0)


In [43]:
# Run static analysis with material properties using config dictionary
results = static_analysis_with_materials(**analysis_config)

STARTING STATIC ANALYSIS (WITH MATERIAL PROPERTIES)

[1] Markers loaded:
    Fixed marker: 'fixed_end' = -7
    Forced marker: 'forced_end' = -6
    Material marker: 'Domain' = -8

[2] Mesh loaded:
    Vertices: 44
    Cells: 60
    Facets: 0

[1] Markers loaded:
    Fixed marker: 'fixed_end' = -7
    Forced marker: 'forced_end' = -6
    Material marker: 'Domain' = -8

[2] Mesh loaded:
    Vertices: 44
    Cells: 60
    Facets: 0

[3] Markers loaded:
    Surface marker values: {0, -7, -6, 2147483647}
    Solid marker values: {-8}

[4] Function space created:
    DOFs: 132

[5] Material properties assigned:
    E = 2.00e+06 Pa, nu = 0.3
    μ = 7.69e+05 Pa, λ = 1.15e+06 Pa

[6] Setting up boundary conditions:
    Looking for: Fixed=-7, Forced=-6
    Fixed facets found: 2
    Forced facets found: 2

[3] Markers loaded:
    Surface marker values: {0, -7, -6, 2147483647}
    Solid marker values: {-8}

[4] Function space created:
    DOFs: 132

[5] Material properties assigned:
    E = 2.00